In [199]:
import os
import requests
import json
from openai import OpenAI
import datetime
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [82]:
def trackEvent(email, scope, groupBy=None, analysis=True):
    url = "http://localhost:3000/track"
    headers = {'Content-Type': 'application/json'}

    data = {
        "orgId": "1",
        "email": email,
        "scope": scope,
        "groupBy": groupBy if groupBy is not None else 'event',
        "analysis": analysis if analysis is not None else True
    }

    response = requests.get(url, headers=headers, data=json.dumps(data))
    return response.json()

In [104]:
def createEvent(email, startTime, endTime, timezone="America/New_York", summary=None, description=None, location=None):
    url = "http://localhost:3000/manage"
    headers = {'Content-Type': 'application/json'}

    data = {
        "orgId": "1",
        "email": email,
        "type": "insert",
        "startTime": startTime,
        "endTime": endTime,
        "timezone": timezone,
        "summary": summary,
        "description": description,
        "location": location
    }

    # Remove keys with None values
    data = {k: v for k, v in data.items() if v is not None}

    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()

# # Example usage
# response_data = createEvent(
#     email="example@example.com",
#     type="insert",
#     startTime="2023-01-01T09:00:00",
#     endTime="2023-01-01T10:00:00",
#     summary="Meeting",
#     description="Discussion about project",
#     location="Office"
# )
# print(response_data)


In [7]:
def updateEvent(email, startTime, endTime, eventId, timezone="America/New_York", summary=None, description=None, location=None):
    url = "http://localhost:3000/manage"
    headers = {'Content-Type': 'application/json'}

    data = {
        "orgId": "1",
        "email": email,
        "type": "update",
        "eventId": eventId,
        "startTime": startTime,
        "endTime": endTime,
        "timezone": timezone,
        "summary": summary,
        "description": description,
        "location": location
    }

    # Remove keys with None values
    # data = {k: v for k, v in data.items() if v is not None}

    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()

In [85]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [118]:
client = OpenAI(api_key=OPENAI_API_KEY)
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_7MGhELp9KrqNsDvFQxzeuj49',
 'created_at': 1701304522,
 'metadata': {},
 'object': 'thread'}

In [87]:
def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [124]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="my email is sw3709@columbia.edu"
)

In [123]:
datetime = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= f"Current date and time is {datetime}"
)

In [176]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="help me get the id of my event for tomorrow 9pm and delete that event for me"
)

In [177]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id="asst_2x9WVVd9cMQObO4gFsMhIrbC",
  instructions="Use the approprite function tool to achieve user's specific request."
)

In [178]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(2)
    return run

In [179]:
run = wait_on_run(run, thread)
run.status
# show_json(run)

'requires_action'

In [180]:
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

In [181]:
pretty_print(get_response(thread))

# Messages
user: my email is sw3709@columbia.edu
user: Current date and time is 11/29/2023, 19:39:56
user: my email is sw3709@columbia.edu
user: Create a new event for 8pm tomorrow with the title 'Meeting' and description 'Discuss project'
assistant: A new event titled 'Meeting' with the description 'Discuss project' has been successfully created for 8pm tomorrow. You can view or edit the event using the following [link](https://www.google.com/calendar/event?eid=bTZpZnBsZWY2ODJ2ZTRqcmg0aHNyb3Nucjggc3czNzA5QGNvbHVtYmlhLmVkdQ).
user: Create am event for 9pm tomorrow with the title 'Meeting' and description 'Discuss project'
assistant: Another new event titled 'Meeting' with the description 'Discuss project' has been successfully created for 9pm tomorrow. You can view or edit the event using the following [link](https://www.google.com/calendar/event?eid=ODE4bmF2ZzhodWgyNXBjdDZsOWRrcjFvZDggc3czNzA5QGNvbHVtYmlhLmVkdQ).
user: help me get the id of my event for tomorrow 9pm and delete that ev

In [182]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: deleteEvent
Function Arguments:


{'email': 'sw3709@columbia.edu', 'eventId': '818navg8huh25pct6l9dkr1od8'}

In [185]:
responses = globals()[name](**arguments)
print("Responses:", responses)
# form json object from arguments
# json.dumps(arguments)


Responses: {'type': 'delete', 'eventDetails': {'kind': 'calendar#event', 'etag': '"3402614304210000"', 'id': '818navg8huh25pct6l9dkr1od8', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=ODE4bmF2ZzhodWgyNXBjdDZsOWRrcjFvZDggc3czNzA5QGNvbHVtYmlhLmVkdQ', 'created': '2023-11-30T01:19:12.000Z', 'updated': '2023-11-30T01:19:12.105Z', 'summary': 'Meeting', 'description': 'Discuss project', 'creator': {'email': 'sw3709@columbia.edu', 'self': True}, 'organizer': {'email': 'sw3709@columbia.edu', 'self': True}, 'start': {'dateTime': '2023-11-30T21:00:00-05:00', 'timeZone': 'America/New_York'}, 'end': {'dateTime': '2023-11-30T22:00:00-05:00', 'timeZone': 'America/New_York'}, 'iCalUID': '818navg8huh25pct6l9dkr1od8@google.com', 'sequence': 0, 'reminders': {'useDefault': True}, 'eventType': 'default'}}


In [186]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

{'id': 'run_v7aDRgDTqmAII7itLktEgTRw',
 'assistant_id': 'asst_2x9WVVd9cMQObO4gFsMhIrbC',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1701308646,
 'expires_at': 1701309246,
 'failed_at': None,
 'file_ids': [],
 'instructions': "Use the approprite function tool to achieve user's specific request.",
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1701308646,
 'status': 'queued',
 'thread_id': 'thread_7MGhELp9KrqNsDvFQxzeuj49',
 'tools': [{'function': {'name': 'trackEvent',
    'parameters': {'type': 'object',
     'properties': {'email': {'type': 'string',
       'description': 'The email address associated with the event.'},
      'scope': {'type': 'integer',
       'description': 'An integer representing the scope of the event, between 0 and 30.'},
      'groupBy': {'type': 'string',
       'enum': ['event', 'eventType', 'color'],
       'description': 'A string to specify how the tr

In [191]:
run = wait_on_run(run, thread)
run.status

'completed'

In [188]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_JlRcvbJ1yoom2z9Xe04fsN37', assistant_id='asst_2x9WVVd9cMQObO4gFsMhIrbC', content=[MessageContentText(text=Text(annotations=[], value='The event scheduled for tomorrow at 9pm has been successfully deleted. You no longer have this event in your calendar.'), type='text')], created_at=1701308726, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_v7aDRgDTqmAII7itLktEgTRw', thread_id='thread_7MGhELp9KrqNsDvFQxzeuj49'), ThreadMessage(id='msg_hVK1S5ZXkbawKVq35yk3ELBT', assistant_id='asst_2x9WVVd9cMQObO4gFsMhIrbC', content=[MessageContentText(text=Text(annotations=[], value='The ID of your event for tomorrow at 9pm is `818navg8huh25pct6l9dkr1od8`. I will now proceed to delete this event for you.'), type='text')], created_at=1701308647, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_v7aDRgDTqmAII7itLktEgTRw', thread_id='thread_7MGhELp9KrqNsDvFQxzeuj49'), ThreadMessage(id='ms

In [197]:
# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="desc", after=message.id
)
# messages.data[0].content[0].text.value
pretty_print(get_response(thread))


# Messages
user: my email is sw3709@columbia.edu
user: Current date and time is 11/29/2023, 19:39:56
user: my email is sw3709@columbia.edu
user: Create a new event for 8pm tomorrow with the title 'Meeting' and description 'Discuss project'
assistant: A new event titled 'Meeting' with the description 'Discuss project' has been successfully created for 8pm tomorrow. You can view or edit the event using the following [link](https://www.google.com/calendar/event?eid=bTZpZnBsZWY2ODJ2ZTRqcmg0aHNyb3Nucjggc3czNzA5QGNvbHVtYmlhLmVkdQ).
user: Create am event for 9pm tomorrow with the title 'Meeting' and description 'Discuss project'
assistant: Another new event titled 'Meeting' with the description 'Discuss project' has been successfully created for 9pm tomorrow. You can view or edit the event using the following [link](https://www.google.com/calendar/event?eid=ODE4bmF2ZzhodWgyNXBjdDZsOWRrcjFvZDggc3czNzA5QGNvbHVtYmlhLmVkdQ).
user: help me get the id of my event for tomorrow 9pm and delete that ev

In [184]:
def deleteEvent(email, eventId):
    url = "http://localhost:3000/manage"
    headers = {'Content-Type': 'application/json'}

    data = {
        "orgId": "1",
        "email": email,
        "type": "delete",
        "eventId": eventId
    }

    # Remove keys with None values
    # data = {k: v for k, v in data.items() if v is not None}

    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()

In [ ]:
trackEvent_json = {
    "name": "trackEvent",
    "description": "Sends a tracking event to a specified endpoint and returns the server's response. The function makes a GET request to the specified URL with the provided data.",
    "parameters": {
        "type": "object",
        "properties": {
            # "orgId": {
            #     "type": "string",
            #     "description": "This is a system value and is always 1."
            # },
            "email": {
                "type": "string",
                "description": "The email address associated with the event." # can be found in previous message
            },
            "scope": {
                "type": "integer",
                "description": "An integer representing the scope of the event, between 0 and 30."
            },
            "groupBy": {
                "type": "string",
                "enum": ["event", "eventType", "color"],
                "description": "A string to specify how the tracking data should be grouped."
            },
            "analysis": {
                "type": "boolean",
                "description": "A boolean flag to indicate whether analysis is required for the tracking event."
            }
        },
        "required": ["email", "scope", "analysis"]
    }
    # "returns": {
    #     "type": "object",
    #     "description": "The response from the server, typically in JSON format."
    # }
}


In [ ]:
createEvent_json = {
    "name": "createEvent",
    "description": "Creates an event by sending a POST request with event details to a specified endpoint.",
    "parameters": {
        "type": "object",
        "properties": {
            # "orgId": {
            #     "type": "string",
            #     "description": "This is a system value and is always 1."
            # },
            "email": {
                "type": "string",
                "description": "Email address associated with the event creation."
            },
            # "type": {
            #     "type": "string",
            #     "description": "Type of the event action, This value is always 'insert' for this function."
            # },
            "startTime": {
                "type": "string",
                "description": "Start time of the event in ISO 8601 format."
            },
            "endTime": {
                "type": "string",
                "description": "End time of the event in ISO 8601 format."
            },
            "timezone": {
                "type": "string",
                "default": "America/New_York",
                "description": "Timezone of the event. Default is 'America/New_York'."
            },
            "summary": {
                "type": "string",
                "description": "A brief summary or title of the event."
            },
            "description": {
                "type": "string",
                "description": "Detailed description of the event."
            },
            "location": {
                "type": "string",
                "description": "Physical location or address of the event."
            }
        },
        "required": ["email", "startTime", "endTime"]
    }
}


In [ ]:
updateEvent_json = {
    "name": "updateEvent",
    "description": "Updates an event by sending a POST request with event details to a specified endpoint.",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "Email address associated with the event creation."
            },
            # "type": {
            #     "type": "string",
            #     "description": "Type of the event action, This value is always 'update' for this function."
            # },
            "startTime": {
                "type": "string",
                "description": "Start time of the event in ISO 8601 format."
            },
            "endTime": {
                "type": "string",
                "description": "End time of the event in ISO 8601 format."
            },
            "eventId": {
                "type": "string",
                "description": "ID of the event to be updated."
            },
            "timezone": {
                "type": "string",
                "default": "America/New_York",
                "description": "Timezone of the event. Default is 'America/New_York'."
            },
            "summary": {
                "type": "string",
                "description": "A brief summary or title of the event."
            },
            "description": {
                "type": "string",
                "description": "Detailed description of the event."
            },
            "location": {
                "type": "string",
                "description": "Physical location or address of the event."
            }
        },
        "required": ["email", "eventId"]
    }
}


In [ ]:
deleteEvent_json = {
    "name": "deleteEvent",
    "description": "Deletes an event by sending a POST request with event details to a specified endpoint.",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "Email address associated with the event creation."
            },
            "type": {
                "type": "string",
                "description": "Type of the event action, This value is always 'update' for this function."
            },
            "eventId": {
                "type": "string",
                "description": "ID of the event to be updated."
            },
        },
        "required": ["email", "eventId"]
    }
}
